In [1]:
import tskit, msprime, pyslim
from IPython.display import SVG, display
import numpy as np
import os, subprocess, statistics
#import matplotlib.pyplot as plt

In [4]:
print("BurnInType","ScalingFactor","GenomeSize","RepNum","AvgMissCoalEvents","PropMissBranchLen","AvgMissCoalEvents_4N","PropMissBranchLen_4N", sep='\t')
for t in [5, 10, 20]:
    for c in [1000, 500, 100]:
        N=652700/c
        rr=2.06e-8*c
        for G in ["1e5", "1e6", "1e7"]:
            for n in range(1,3):

                #ts = tskit.load(f'C:\\Users\\tferrari\\Desktop\\SlimBenchmark\\fly\\out\\burnins\\flyBench_burnin{n}_{t}Ne_c{c}_{G}.trees')
                ts = tskit.load(f'C:\\Users\\tferrari\\Desktop\\SlimBenchmark\\fly\\out\\burnins_wselection\\flyBench_burnin{n}_{t}Ne_c{c}_{G}_wsel.trees')
                
                # Measure ts stats per position
                events_for_pos = np.zeros(int(ts.sequence_length)) 
                length_for_pos = np.zeros(int(ts.sequence_length))
                for tree in ts.trees():
                    left, right = map(int, tree.interval)
                    events_for_pos[left: right] = tree.num_roots-1 # Number of missing coalescence events
                    length_for_pos[left: right] = tree.total_branch_length # Tree length
                    
                # Decapitate tree at time = 4N
                dts = ts.decapitate(time = (4*N))
                
                # Measure decapitated ts stats per position
                dts_events_for_pos = np.zeros(int(dts.sequence_length))
                dts_length_for_pos = np.zeros(int(dts.sequence_length))
                for tree in dts.trees():
                    left, right = map(int, tree.interval)
                    dts_events_for_pos[left: right] = tree.num_roots-1
                    dts_length_for_pos[left: right] = tree.total_branch_length
                
                # Recapitate tree
                rts = pyslim.recapitate(ts,
                                        recombination_rate=rr,
                                        ancestral_Ne=N)
                assert(max([t.num_roots for t in rts.trees()]) == 1)
                
                # Measure fully coalesced tree stats per position
                rts_length_for_pos = np.zeros(int(rts.sequence_length))
                for tree in rts.trees():
                    length = tree.total_branch_length
                    left, right = map(int, tree.interval)
                    rts_length_for_pos[left: right] = length
                
                # Summarize stats over all positions
                avg_events = statistics.mean(events_for_pos) # Num missing coalescence events averaged across positions
                prop_length = 1 - sum(length_for_pos)/sum(rts_length_for_pos) # Proportion of missing ts branch length

                # Summarize stats at Time=4N
                T4N_avg_events = statistics.mean(dts_events_for_pos) # Num missing coalescence events averaged across positions
                T4N_prop_length = 1 - sum(dts_length_for_pos)/sum(rts_length_for_pos) # Proportion of missing ts branch length
                
                # Print
                print(t,c,G,n,avg_events,prop_length,T4N_avg_events,T4N_prop_length,sep='\t')

BurnInType	ScalingFactor	GenomeSize	RepNum	AvgMissCoalEvents	PropMissBranchLen	AvgMissCoalEvents_4N	PropMissBranchLen_4N
5	1000	1e5	1	0.02521	0.0037675477325910256	0.07202	0.046990750460761954
5	1000	1e5	2	0.02894	0.0043794052916359405	0.07255	0.04838358882226146
5	1000	1e6	1	0.0	0.0	0.0	0.08591031908870594
5	1000	1e6	2	0.0	0.0	0.0	0.08727403759143271
5	1000	1e7	1	0.0	0.0	0.0	0.1345185317758535
5	1000	1e7	2	0.0	0.0	0.0	0.13391307109989548
5	500	1e5	1	0.05475	0.007342062436425212	0.13008	0.04494647126708795
5	500	1e5	2	0.05058	0.0076456122786833225	0.11305	0.04486684016060194
5	500	1e6	1	0.001267	0.00029554686208332903	0.006812	0.04974315971697607
5	500	1e6	2	0.001593	0.00033429496812753534	0.008209	0.05021130731616896
5	500	1e7	1	0.0	0.0	0.0	0.11029806634530526
5	500	1e7	2	0.0	0.0	0.0	0.11019220276349817
5	100	1e5	1	0.07064	0.007758119332484159	0.14722	0.03849389883215948
5	100	1e5	2	0.06989	0.007302295038627116	0.14809	0.03793133309257568
5	100	1e6	1	0.048443	0.005538519301004685	0.11